<a href="https://colab.research.google.com/github/jeromenicholas07/FlowerIdentifier/blob/master/FlowerClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import cv2
import matplotlib.pyplot as plt

In [0]:
from google.colab import drive

drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
datadir = "/content/drive/My Drive/Flower"
valdir = "/content/drive/My Drive/Flowers_valid"
categories = ["Angels Trumpet","Beach Moonflower","Bougainvillea","Cyclamen","Daisy","Dandelion","Foxgloves","Frangipani","Gladiolus","Hawaiian Hibiscus","Nasturtium","Purple Passionflower","Rose","Sacred Lotus","Sunflower","Tulip","Wallflowers"]
img_size = 200

In [0]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 1s 0us/step


In [0]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0x7f47b6091390> False
<keras.layers.convolutional.Conv2D object at 0x7f4768d612e8> False
<keras.layers.convolutional.Conv2D object at 0x7f4768d61198> False
<keras.layers.pooling.MaxPooling2D object at 0x7f4768507b00> False
<keras.layers.convolutional.Conv2D object at 0x7f47685075c0> False
<keras.layers.convolutional.Conv2D object at 0x7f47684bd400> False
<keras.layers.pooling.MaxPooling2D object at 0x7f47684ccef0> False
<keras.layers.convolutional.Conv2D object at 0x7f47684ccba8> False
<keras.layers.convolutional.Conv2D object at 0x7f476847fa20> False
<keras.layers.convolutional.Conv2D object at 0x7f4768493cf8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f4768443438> False
<keras.layers.convolutional.Conv2D object at 0x7f4768443a58> False
<keras.layers.convolutional.Conv2D object at 0x7f47683ee668> False
<keras.layers.convolutional.Conv2D object at 0x7f47684046a0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f47683ae518>

In [0]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(17, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 6, 6, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              18875392  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 17)                17425     
Total params: 33,607,505
Trainable params: 33,607,505
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
 


In [0]:
train_data_dir = '/content/drive/My Drive/Flower'
validation_data_dir = '/content/drive/My Drive/Flowers_valid'

In [0]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2, 
      horizontal_flip=True,
      fill_mode='nearest')

In [0]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_batchsize = 100
val_batchsize = 10

In [0]:
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_size, img_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_size, img_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 2795 images belonging to 17 classes.
Found 269 images belonging to 17 classes.


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
28/27 [==============================] - 757s 27s/step - loss: 2.8791 - acc: 0.0601 - val_loss: 2.8273 - val_acc: 0.0669
Epoch 2/30
28/27 [==============================] - 34s 1s/step - loss: 2.8250 - acc: 0.0827 - val_loss: 2.7991 - val_acc: 0.0892
Epoch 3/30
28/27 [==============================] - 42s 1s/step - loss: 2.7947 - acc: 0.0973 - val_loss: 2.7467 - val_acc: 0.0706
Epoch 4/30
28/27 [==============================] - 44s 2s/step - loss: 2.6800 - acc: 0.1361 - val_loss: 2.9617 - val_acc: 0.1524
Epoch 5/30
28/27 [==============================] - 43s 2s/step - loss: 2.6255 - acc: 0.1613 - val_loss: 2.6287 - val_acc: 0.1673
Epoch 6/30
28/27 [==============================] - 46s 2s/step - loss: 2.3366 - acc: 0.2414 - val_loss: 2.0710 - val_acc: 0.3941
Epoch 7/30
28/27 [==============================] - 43s 2s/step - loss: 2.1680 - acc: 0.3319 - val_loss: 2.7440 - val_acc: 0.3086
Epoch 8/30
28/27 [======================

In [0]:
model.save('model.h5')

In [0]:
from tensorflow.contrib import lite
converter = lite.TFLiteConverter.from_keras_model_file( 'model.h5' ) # Your model's name
converter.post_training_quantize=True
model = converter.convert()
file = open( 'q_model.tflite' , 'wb' ) 
file.write( model )

INFO:tensorflow:Froze 30 variables.
INFO:tensorflow:Converted 30 variables to const ops.


33629960

In [0]:
from keras.models import Model
from keras.layers import *
import os
import tensorflow as tf

In [0]:
def keras_to_tensorflow(keras_model, output_dir, model_name,out_prefix="output_", log_tensorboard=True):

    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)

    out_nodes = []

    for i in range(len(keras_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(keras_model.output[i], out_prefix + str(i + 1))

    sess = K.get_session()

    from tensorflow.python.framework import graph_util, graph_io

    init_graph = sess.graph.as_graph_def()

    main_graph = graph_util.convert_variables_to_constants(sess, init_graph, out_nodes)

    graph_io.write_graph(main_graph, output_dir, name=model_name, as_text=False)

    if log_tensorboard:
        from tensorflow.python.tools import import_pb_to_tensorboard

        import_pb_to_tensorboard.import_to_tensorboard(
            os.path.join(output_dir, model_name),
            output_dir)


In [0]:
output_dir = '/content/'

In [0]:
keras_to_tensorflow(model,output_dir=output_dir,model_name="saved_model.pb")

INFO:tensorflow:Froze 30 variables.
INFO:tensorflow:Converted 30 variables to const ops.
Model Imported. Visualize by running: tensorboard --logdir=/content/


In [0]:
interpreter = tf.contrib.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
# change the following line to feed into your own data.
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[8.4829348e-12 3.5071002e-13 1.5394824e-13 9.1178454e-10 8.7332595e-03
  1.2238419e-02 3.0457886e-10 4.8573139e-11 8.7097683e-11 2.9146777e-10
  1.1180977e-11 9.6238639e-10 4.5035500e-09 6.6240183e-08 2.3583577e-06
  9.7902590e-01 2.5022080e-13]]


RuntimeError: ignored

In [0]:
import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="q_model.tflite")
interpreter.allocate_tensors()

# Print input shape and type
print(interpreter.get_input_details()[0])  # Example: [1 224 224 3]
print(interpreter.get_input_details()[0]['dtype'])  # Example: <class 'numpy.float32'>

# Print output shape and type
print(interpreter.get_output_details()[0])  # Example: [1 1000]
print(interpreter.get_output_details()[0]['dtype'])

{'name': 'vgg16_input', 'index': 51, 'shape': array([  1, 200, 200,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
<class 'numpy.float32'>
{'name': 'dense_2/Softmax', 'index': 18, 'shape': array([ 1, 17], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
<class 'numpy.float32'>
